In [ ]:
%run eda.ipynb import *

In [3]:
import feature_selector as fs
import data_loader as dl

In [6]:
data_path = "./data/dataset"
participant_ids = list(range(88))
random.shuffle(participant_ids)

train_ids = participant_ids[:80]
test_ids = participant_ids[80:]

train_loader = DataLoader(data_path, train_ids)
test_loader = DataLoader(data_path, test_ids)

selector1 = fs.BaselineSelector()
selector1.selectFeatures(['mean', 'kurtosis'])
out1 = selector1.transform(arr)